## Search SL in LsstComCam

In [ ]:
from lsst.daf.butler import Butler
from lsst.geom import SpherePoint, degrees
import lsst.afw.display as afwDisplay
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from lsst.geom import SpherePoint, degrees

In [ ]:
afwDisplay.setDefaultBackend("firefly")
framecount = 1

In [ ]:
def find_tract_patch_and_display(butler, skymap,ra_deg, dec_deg, band, framecount):
    # Ouvrir le dépôt
    #butler = Butler(repo)
    #skymap = butler.get("skyMap")
    band_to_color = {"u":"blue","g":"green","r":"red","i":"orange","z":"yellow","y":"purple"}

    datasettype = "objectTable_tract"
    therefs = butler.registry.queryDatasets(datasettype,  collections=collection)

    
    # Créer le point céleste
    coord = SpherePoint(ra_deg * degrees, dec_deg * degrees)

    # Trouver tract/patch
#    for tractInfo in skymap:
#        if tractInfo.contains(coord):
#            patchInfo = tractInfo.findPatch(coord)
#            tract = tractInfo.tract
#            patch = patchInfo.getIndex()  # Tuple (x, y)
#            print(f"Tract: {tract}, Patch: {patch}")
#            break
#    else:
#        print("Coordonnée en dehors du skymap.")
#.   return
    found = False
    # loop only on existing tracts
    
    for ref in therefs:
        tract_id = ref.dataId["tract"]
        tractInfo = skymap[tract_id]
        if tractInfo.contains(coord):
            patchInfo = tractInfo.findPatch(coord)
            patch_id = patchInfo.getIndex()  # Tuple (x, y)
            patch_indexseq = patchInfo.getSequentialIndex()
            print(f"Tract: {tract_id}, Patch: {patch_indexseq}")
            found = True
            break
    # Charger l’image coadd correspondante
    dataId = {"tract": tract_id, "patch": patch_indexseq, "band": band,"skymap": skymapName}
    #deepCoadd_calexp
    coadd = butler.get("deepCoadd_calexp", dataId)
    # Récupération du WCS
    wcs = coadd.getWcs()

    datatitle = f"deepcoadd-LSSTComCam : {tract_id} , {patch_indexseq} , {band}"
    print(datatitle)

    # Affichage avec marqueur
    
    display = afwDisplay.Display(frame = framecount)
    display.scale('asinh', 'zscale')
    #afw_display.scale('linear', min=zmin, max=zmax)
    #display.setImageColormap(cmap='grey')
    display.setImageColormap(cmap='Grey_r')
    display.mtv(coadd.getImage(),title=datatitle)

    # Création du point céleste
    pix_point = wcs.skyToPixel(coord)
    x, y = pix_point.x,pix_point.y
    
    col = band_to_color[band] 
    display.dot("+", x, y, size=50, ctype=col)
    

    
    return tract_id, patch_indexseq,band

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666' # work
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810' # work
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359' # work


# bad : crash collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'

# bad : collection = "LSSTComCam/runs/DRP/20241101_20241211/w_2024_51/DM-48233"

# not working perhaps because I am using w_2025_10 version
# bad : no ccd visit collection = "LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864"
# bad : no ccd visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_15/DM-50050'
# bad : no cce visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864'
# bad : no cce visit collection collection = 'LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751'


instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = \'" + instrument+ "\'"
collectionStr = collection.replace("/", "_")
NDET = 9
TRACTSEL = 5063

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry
skymap = butler.get('skyMap', skymap=skymapName, collections=collection)

In [ ]:
# List a number of usefull data-product to explore
if 0:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and ( "deepCoadd" in datasetType.name)
            ):
                print(datasetType)

In [ ]:
#find_tract_patch_and_display(repo, ra, dec, band="i")

## Definition of SL in ECDFS

In [ ]:
sources = {}
sources[15422] = SkyCoord("03 32 38.21 -27 56 53.2", unit = (u.hourangle,u.deg))  
sources[4244] =  SkyCoord("03 32 06.45 -27 47 28.6",  unit = (u.hourangle,u.deg)) 
sources[40173] = SkyCoord("03 33 19.45 -27 44 50.0", unit = (u.hourangle,u.deg)) 

In [ ]:
sources[43242] = SkyCoord("03 31 55.35 -27 43 23.5", unit = (u.hourangle,u.deg)) 
sources[46446] = SkyCoord("03 31 35.94 -27 41 48.2", unit = (u.hourangle,u.deg))  

## Check the DeepCoadds

In [ ]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [ ]:
datasettype = "deepCoadd_calexp"
df_tractpatchband = pd.DataFrame(columns = ["tract","patch","band"])
therefs = butler.registry.queryDatasets(datasettype,  collections=collection)
for count,ref in enumerate(therefs):
    df_tractpatchband.loc[count] =  [int(ref.dataId["tract"]), int(ref.dataId["patch"]), ref.dataId["band"]]

In [ ]:
df_tractpatchband.sort_values(by=['tract', 'patch','band'],inplace=True)
df_tractpatchband.reset_index(drop=True,inplace=True)

In [ ]:
df_tractpatchband[df_tractpatchband.tract == TRACTSEL]

In [ ]:
dataId = {"tract": TRACTSEL, "patch": 0, "band": "g","skymap": skymapName}
#deepCoadd_calexp
coadd = butler.get("deepCoadd_calexp", dataId)

In [ ]:
coadd.image

In [ ]:
#display = afwDisplay.Display(frame=2)
#display.scale("asinh", "zscale")
#display.mtv(coadd.image)

In [ ]:
#wcs = coadd.getWcs()

## Get images

In [ ]:
num = 15422

In [ ]:
ra_deg = float(sources[num].ra.degree)
dec_deg = float(sources[num].dec.degree)

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype,  collections=collection)
tractsId_list = np.unique([float(ref.dataId['tract']) for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

In [ ]:
# Créer le point céleste
coord = SpherePoint(ra_deg * degrees, dec_deg * degrees)
found = False
for ref in therefs:
    tract_id = ref.dataId["tract"]
    tractInfo = skymap[tract_id]
    if tractInfo.contains(coord):
        patchInfo = tractInfo.findPatch(coord)
        patch = patchInfo.getIndex()  # Tuple (x, y)
        print(f"Tract: {tract_id}, Patch: {patch}")
        found = True
        break

In [ ]:
all_bands = ["g","r","i","y"]

framecount = 0
for iband,band in enumerate(all_bands):
    framecount +=1
    find_tract_patch_and_display(butler, skymap,ra_deg, dec_deg, band,framecount=framecount)